In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function,convert_python_function_to_openai_function,convert_pydantic_to_openai_function

In [ ]:
# https://github.com/OpenBB-finance/OpenBBTerminal/blob/develop/openbb_platform/openbb/assets/reference.json

In [ ]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [1]:
# from openapi_core import OpenAPI

# openapi = OpenAPI.from_file_path("reference.json")
import json

with open("reference.json","r") as file:
    data = json.load(file)

In [2]:
data.keys()

dict_keys(['openbb', 'info', 'paths', 'routers'])

In [57]:
openbb_functions = []

for paths in data['paths']:
    curr_func = {}
    curr_func['name'] = " ".join(paths.split("/")[1:])
    curr_func['description'] = data["paths"][paths]['description']
    curr_func['examples'] = data["paths"][paths]['examples']
    curr_func['parameters'] = {"type":"object","properties":{},"required":[]}
    params_dict = data["paths"][paths]['parameters']
    required_params = []
    for params in params_dict:
        for params_desc in params_dict[params]:
            prop_name = params_desc['name']
            curr_func['parameters']['properties'][prop_name] = params_desc
            if "optional" in params_desc:
                if not params_desc["optional"]:
                    required_params.append(params_desc["name"])
    curr_func['parameters']['required'] = required_params
    openbb_functions.append(curr_func)
    # break
        
    
    # print(data['paths'][paths]['model'])
    # print(data["paths"][paths]['parameters'].keys())
    # break

In [58]:
len(openbb_functions)

122

In [59]:
openbb_functions[1]

{'name': 'crypto search',
 'description': 'Search available cryptocurrency pairs within a provider.',
 'examples': "\nExamples\n--------\n\n```python\nfrom openbb import obb\nobb.crypto.search(provider='fmp')\nobb.crypto.search(query='BTCUSD', provider='fmp')\n```\n\n",
 'parameters': {'type': 'object',
  'properties': {'query': {'name': 'query',
    'type': 'str',
    'description': 'Search query.',
    'default': None,
    'optional': True},
   'provider': {'name': 'provider',
    'type': "Literal['fmp']",
    'description': "The provider to use for the query, by default None. If None, the provider specified in defaults is selected or 'fmp' if there is no default.",
    'default': 'fmp',
    'optional': True}},
  'required': []}}

## STEPS
1. separate by provider and it's arguments

In [19]:
import re
import json

with open("reference.json","r") as file:
    data = json.load(file)
openbb_functions_enum = []

for paths in data['paths']:
    curr_func = {}
    func_name = paths.split("/")[1:]
    func_name = "obb_" + "_".join(func_name)
    curr_func['name'] = func_name
    curr_func['description'] = data["paths"][paths]['description']
    # curr_func['examples'] = data["paths"][paths]['examples']
    curr_func['parameters'] = {"type":"object","properties":{},"required":[]}
    params_dict = data["paths"][paths]['parameters']
    required_params = []
    # print(paths)
    # Providers
    function_options = list(params_dict.keys())

    for params in params_dict:
        # params can be standard or other options
        for params_desc in params_dict[params]:
            prop_name = params_desc['name']
            
            if "type" in params_desc:
                if "None" in params_desc['type'] and "Literal" in params_desc['type']:
                    result = re.findall(r"Literal\[(.*?)\]", params_desc['type'])[0].split(", ")
                    result = [value.strip('"') if value.strip() != "None" else None for value in result]    
                    result = [value.replace("'", '') for value in result if value is not None]
                    del params_desc['type']
                    for res in result:
                        if res in function_options:
                            params_desc['optional'] = False
                            break
                    params_desc.update({"type":"string","enum":result})
                elif "None" not in params_desc['type'] and "Literal" in params_desc['type']:
                    result = re.findall(r"Literal\[(.*?)\]", params_desc['type'])[0].split(", ")
                    result = [value.replace("'", '') for value in result] 
                    for res in result:
                        if res in function_options:
                            params_desc['optional'] = False
                            break
                    del params_desc['type']
                    params_desc.update({"type":"string","enum":result})
                elif "int" in params_desc['type']:
                    params_desc['type'] = "integer"
                elif ("Union" in params_desc['type'] or "List" in params_desc['type']) and "str" in params_desc['type']:
                    params_desc['type'] = "string"
                elif "str" in params_desc['type']:
                    params_desc['type'] = "string"
            curr_func['parameters']['properties'][prop_name] = params_desc
            if "optional" in params_desc:
                if not params_desc["optional"] and prop_name not in required_params:
                    required_params.append(prop_name)
            del params_desc['name']
    curr_func['parameters']['required'] = required_params
    openbb_functions_enum.append(curr_func)
    # break

In [21]:
len(openbb_functions_enum)

122

## METHOD

1. standard + one provider --> have one function only that has provider equal to the given one
2. Standard + multiple provider --> have multiple functions, one for standard and other for each provider

In [9]:
def process_params(params_desc):
            
    if "type" in params_desc:
        if "None" in params_desc['type'] and "Literal" in params_desc['type']:
            result = re.findall(r"Literal\[(.*?)\]", params_desc['type'])[0].split(", ")
            result = [value.strip('"') if value.strip() != "None" else None for value in result]    
            result = [value.replace("'", '') for value in result if value is not None]
            del params_desc['type']
            # for res in result:
                # if res in function_options:
            #         params_desc['optional'] = False
            #         break
            params_desc.update({"type":"string","enum":result})
        elif "None" not in params_desc['type'] and "Literal" in params_desc['type']:
            result = re.findall(r"Literal\[(.*?)\]", params_desc['type'])[0].split(", ")
            result = [value.replace("'", '') for value in result] 
            # for res in result:
            #     if res in function_options:
            #         params_desc['optional'] = False
            #         break
            del params_desc['type']
            params_desc.update({"type":"string","enum":result})
        elif "int" in params_desc['type']:
            params_desc['type'] = "integer"
        elif ("Union" in params_desc['type'] or "List" in params_desc['type']) and "str" in params_desc['type']:
            params_desc['type'] = "string"
        elif "str" in params_desc['type']:
            params_desc['type'] = "string"
    return params_desc
def get_curr_func(data,paths,ignore_non_standard:bool=False):
    curr_func = {}
    func_name = paths.split("/")[1:]
    func_name = "obb_" + "_".join(func_name)
    curr_func['name'] = func_name
    curr_func['description'] = data["paths"][paths]['description']
    # curr_func['examples'] = data["paths"][paths]['examples']
    curr_func['parameters'] = {"type":"object","properties":{},"required":[]}
    required_params = []
    # print(paths)
    # Providers
    for params in params_dict:
    # params can be standard or other options
        if ignore_non_standard and params!="standard": break
        for params_desc in params_dict[params]:
            prop_name = params_desc['name']
            params_desc = process_params(params_desc)
            curr_func['parameters']['properties'][prop_name] = params_desc
            if "optional" in params_desc:
                if not params_desc["optional"] and prop_name not in required_params:
                    required_params.append(prop_name)
                del params_desc["optional"]
            del params_desc['name']
    curr_func['parameters']['required'] = required_params + ['provider']
    return curr_func

In [13]:
import re
import json
from copy import deepcopy

with open("reference.json","r") as file:
    data = json.load(file)
openbb_functions_enum = []
funcs = 0
for paths in data['paths']:
    params_dict = data["paths"][paths]['parameters']
    function_options = list(params_dict.keys())
    if len(function_options) == 2 and function_options[0] == "standard":
        curr_func = get_curr_func(data,paths)
        curr_func['name'] += f"_{function_options[1]}"
        openbb_functions_enum.append(curr_func)
        funcs+=len(function_options)
    elif len(function_options)>2:
        # standard function extraction
        standard_func = get_curr_func(data,paths,ignore_non_standard=True)
        openbb_functions_enum.append(standard_func)
        for fo in function_options[1:]:
            curr_func = deepcopy(standard_func)
            curr_func['name'] = curr_func['name'] + "_" + fo
            curr_func['description'] = f"{curr_func['description']} Get it from provider {fo}"
            provider_extra_params = params_dict[fo]
            # print(provider_extra_params)
            for params_desc in provider_extra_params:
                prop_name = params_desc['name']
                params_desc = process_params(params_desc)
                curr_func['parameters']['properties'][prop_name] = params_desc
            curr_func['parameters']['properties']['provider']['enum'] = [fo]
            # curr_func['parameters']['required']
            openbb_functions_enum.append(curr_func)
        funcs+=len(function_options)

In [14]:
openbb_functions_enum[-2]

{'name': 'obb_regulators_sec_rss_litigation_sec',
 'description': 'Get the RSS feed that provides links to litigation releases concerning civil lawsuits brought by the Commission in federal court.',
 'parameters': {'type': 'object',
  'properties': {'provider': {'description': "The provider to use for the query, by default None. If None, the provider specified in defaults is selected or 'sec' if there is no default.",
    'default': 'sec',
    'type': 'string',
    'enum': ['sec']}},
  'required': ['provider']}}

In [15]:
json_string = json.dumps(openbb_functions_enum)

# Specify the file path where you want to save the JSON data
file_path = "openbb_functions_enum.json"

# Open the file in write mode
with open(file_path, "w") as file:
    # Write the JSON string to the file
    file.write(json_string)

In [13]:
from openbb import obb
obb.crypto.price.historical("BTCUSD",provider="yfinance")

OBBject

id: 0661df7c-80c3-76c9-8000-5abc414b389e
results: [{'date': datetime.date(2023, 4, 16), 'open': 30315.9765625, 'high': 30555...
provider: yfinance
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'}, 's...

In [70]:
openbb_functions_enum[0]

{'name': 'obb_crypto_price_historical',
 'description': 'Get historical price data for cryptocurrency pair(s) within a provider.',
 'parameters': {'type': 'object',
  'properties': {'symbol': {'type': 'string',
    'description': 'Symbol to get data for. Can use CURR1-CURR2 or CURR1CURR2 format. Multiple items allowed for provider(s): fmp, polygon, tiingo, yfinance.',
    'default': '',
    'optional': False},
   'start_date': {'type': 'string',
    'description': 'Start date of the data, in YYYY-MM-DD format.',
    'default': None,
    'optional': True},
   'end_date': {'type': 'string',
    'description': 'End date of the data, in YYYY-MM-DD format.',
    'default': None,
    'optional': True},
   'provider': {'description': "The provider to use for the query, by default None. If None, the provider specified in defaults is selected or 'fmp' if there is no default.",
    'default': 'fmp',
    'optional': False,
    'type': 'string',
    'enum': ['fmp', 'polygon', 'tiingo', 'yfinance']

In [85]:
# openbb_functions_enum[0]

In [8]:
openbb_functions_enum[1]

{'name': 'CryptoSearch',
 'description': 'Search available cryptocurrency pairs within a provider.',
 'examples': "\nExamples\n--------\n\n```python\nfrom openbb import obb\nobb.crypto.search(provider='fmp')\nobb.crypto.search(query='BTCUSD', provider='fmp')\n```\n\n",
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string',
    'description': 'Search query.',
    'default': None,
    'optional': True},
   'provider': {'description': "The provider to use for the query, by default None. If None, the provider specified in defaults is selected or 'fmp' if there is no default.",
    'default': 'fmp',
    'optional': True,
    'type': 'string',
    'enum': ['fmp']}},
  'required': []}}

In [66]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import openai
import os

load_dotenv(override=True)
openai.api_key = os.environ["OPENAI_API_KEY"]
model = ChatOpenAI()

In [80]:
openbb_functions_enum[0]

{'name': 'obb_crypto_price_historical',
 'description': 'Get historical price data for cryptocurrency pair(s) within a provider.',
 'parameters': {'type': 'object',
  'properties': {'symbol': {'type': 'string',
    'description': 'Symbol to get data for. Can use CURR1-CURR2 or CURR1CURR2 format. Multiple items allowed for provider(s): fmp, polygon, tiingo, yfinance.',
    'default': '',
    'optional': False},
   'start_date': {'type': 'string',
    'description': 'Start date of the data, in YYYY-MM-DD format.',
    'default': None,
    'optional': True},
   'end_date': {'type': 'string',
    'description': 'End date of the data, in YYYY-MM-DD format.',
    'default': None,
    'optional': True},
   'provider': {'description': "The provider to use for the query, by default None. If None, the provider specified in defaults is selected or 'fmp' if there is no default.",
    'default': 'fmp',
    'optional': False,
    'type': 'string',
    'enum': ['fmp', 'polygon', 'tiingo', 'yfinance']

In [91]:
model_with_function = model.bind(functions=openbb_functions_enum[:1])

val = model_with_function.invoke("Get the historical price of ETH for the month of January 2024 from yahoo finance?")

In [92]:
val

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"symbol":"ETH-USD","start_date":"2024-01-01","end_date":"2024-01-31","provider":"yfinance","interval":"1d"}', 'name': 'obb_crypto_price_historical'}}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 409, 'total_tokens': 459}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'function_call', 'logprobs': None}, id='run-93588af4-b5d9-48d7-af74-3a539410e077-0')

In [93]:
import ast
d = ast.literal_eval(val.additional_kwargs['function_call']['arguments'])
# d['provider'] = 'yfinance'

In [94]:
d

{'symbol': 'ETH-USD',
 'start_date': '2024-01-01',
 'end_date': '2024-01-31',
 'provider': 'yfinance',
 'interval': '1d'}

In [43]:
import ast

obb.crypto.price.historical(**d)

OBBject

id: 0661dfa8-933f-7041-8000-ce673e61d5ed
results: [{'date': datetime.date(2021, 1, 1), 'open': 28994.009765625, 'high': 2960...
provider: yfinance
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'}, 's...

In [36]:
val

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"symbol":"BTC-USD","start_date":"2021-01-01","end_date":"2022-01-01","interval":"1d","sort":"asc"}', 'name': 'obb_crypto_price_historical'}}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 403, 'total_tokens': 452}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'function_call', 'logprobs': None}, id='run-b799983e-e3cc-405c-92b9-8dfa739ba25c-0')

In [30]:
openbb_functions_enum[1]

{'name': 'obb_crypto_search',
 'description': 'Search available cryptocurrency pairs within a provider.',
 'examples': "\nExamples\n--------\n\n```python\nfrom openbb import obb\nobb.crypto.search(provider='fmp')\nobb.crypto.search(query='BTCUSD', provider='fmp')\n```\n\n",
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string',
    'description': 'Search query.',
    'default': None,
    'optional': True},
   'provider': {'description': "The provider to use for the query, by default None. If None, the provider specified in defaults is selected or 'fmp' if there is no default.",
    'default': 'fmp',
    'optional': True,
    'type': 'string',
    'enum': ['fmp']}},
  'required': []}}

In [1]:
import json

with open("reference.json","r") as file:
    data = json.load(file)

In [17]:
idx = 122
paths_list = list(data['paths'].keys())

path_values = data['paths'][paths_list[idx]]
print("Path")
print(paths_list[idx])
print()
print(path_values['examples'])
print(path_values['description'])

IndexError: list index out of range

In [12]:
from openbb import obb
obb.keys.mykeys(show = True)

AttributeError: 'App' object has no attribute 'keys'

In [9]:
from openbb import obb
obb.economy.fred_search(provider='fred')

OpenBBError: 
Type -> OpenBBError

Detail -> Missing credential 'fred_api_key'. Check https://fred.stlouisfed.org/ to get it.

In [22]:
obb.crypto.price.historical(symbol='SOL-USD',provider="yfinance",interval="1Q").to_dataframe()

,open,high,low,close,volume
date,,,,,
2023-07-01,18.896065,30.057695,17.461477,21.395147,3.746922e+10
2023-10-01,21.392059,125.193893,21.068972,101.505821,1.826848e+11
2024-01-01,101.512810,209.696060,79.066696,202.874130,3.314962e+11
2024-04-01,202.874435,204.179764,119.833344,137.111740,7.362490e+10


In [32]:
obb.currency.price.historical(symbol='JPYUSD', provider='yfinance').to_dataframe(0)

,date,open,high,low,close,volume
0,2023-04-17,0.007464,0.007479,0.007431,0.007464,0.0
1,2023-04-18,0.007439,0.007470,0.007424,0.007439,0.0
2,2023-04-19,0.007460,0.007465,0.007405,0.007460,0.0
3,2023-04-20,0.007424,0.007461,0.007410,0.007424,0.0
4,2023-04-21,0.007463,0.007488,0.007437,0.007463,0.0
...,...,...,...,...,...,...
257,2024-04-10,0.006589,0.006593,0.006539,0.006589,0.0
258,2024-04-11,0.006537,0.006546,0.006523,0.006537,0.0
259,2024-04-12,0.006528,0.006552,0.006520,0.006528,0.0
260,2024-04-15,0.006524,0.006524,0.006475,0.006524,0.0


## RETRIEVER

In [9]:
import re
import json
import ast
from langchain.schema import Document

with open("reference.json","r") as file:
    data = json.load(file)
retrieved_docs = []
for paths in data['paths']:
    curr_func = {}
    func_name = paths.split("/")[1:]
    func_name = "obb_" + "_".join(func_name)
    curr_func['name'] = func_name
    curr_func['description'] = data["paths"][paths]['description']
    # curr_func['examples'] = data["paths"][paths]['examples']
    curr_examples = data["paths"][paths]['examples']
    
    curr_func['parameters'] = {"type":"object","properties":{},"required":[]}
    params_dict = data["paths"][paths]['parameters']
    required_params = []
    # print(paths)
    function_options = list(params_dict.keys())
    for params in params_dict:
        # params can be standard or other options
        for params_desc in params_dict[params]:
            prop_name = params_desc['name']
            
            if "type" in params_desc:
                if "None" in params_desc['type'] and "Literal" in params_desc['type']:
                    result = re.findall(r"Literal\[(.*?)\]", params_desc['type'])[0].split(", ")
                    result = [value.strip('"') if value.strip() != "None" else None for value in result]    
                    result = [value.replace("'", '') for value in result if value is not None]
                    del params_desc['type']
                    for res in result:
                        if res in function_options:
                            params_desc['optional'] = False
                            break
                    params_desc.update({"type":"string","enum":result})
                elif "None" not in params_desc['type'] and "Literal" in params_desc['type']:
                    result = re.findall(r"Literal\[(.*?)\]", params_desc['type'])[0].split(", ")
                    result = [value.replace("'", '') for value in result] 
                    for res in result:
                        if res in function_options:
                            params_desc['optional'] = False
                            break
                    del params_desc['type']
                    params_desc.update({"type":"string","enum":result})
                elif "int" in params_desc['type']:
                    params_desc['type'] = "integer"
                elif ("Union" in params_desc['type'] or "List" in params_desc['type']) and "str" in params_desc['type']:
                    params_desc['type'] = "string"
                elif "str" in params_desc['type']:
                    params_desc['type'] = "string"
            curr_func['parameters']['properties'][prop_name] = params_desc
            if "optional" in params_desc:
                if not params_desc["optional"] and prop_name not in required_params:
                    required_params.append(prop_name)
            del params_desc['name']
    curr_func['parameters']['required'] = required_params
    retrieved_docs.append(Document(page_content=data["paths"][paths]['description'] + curr_examples,metadata={"metadata":str(curr_func)}))

In [10]:
from dotenv import load_dotenv
from langchain_chroma import Chroma
# import chromadb.utils.embedding_functions as embedding_functions
from langchain_community.embeddings.openai import OpenAIEmbeddings
load_dotenv(override=True)
import os
import openai

openai_ef = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=os.environ['OPENAI_API_KEY']
)
# docs = [d.page_content for d in retrieved_docs]
# metadata = [d.metadata for d in retrieved_docs]
# db2 = Chroma.from_documents(retrieved_docs, openai_ef, persist_directory="JSON_OPENAI")

In [12]:
openai_ef = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=os.environ['OPENAI_API_KEY']
)
db3 = Chroma(persist_directory="JSON_OPENAI", embedding_function=openai_ef)

db3.similarity_search("Give me the historical price of Bitcoin cryptocurrency")

[Document(page_content="Currency Historical Price. Currency historical data.\n\nCurrency historical prices refer to the past exchange rates of one currency against\nanother over a specific period.\nThis data provides insight into the fluctuations and trends in the foreign exchange market,\nhelping analysts, traders, and economists understand currency performance,\nevaluate economic health, and make predictions about future movements.\nExamples\n--------\n\n```python\nfrom openbb import obb\nobb.currency.price.historical(symbol='EURUSD', provider='fmp')\n# Filter historical data with specific start and end date.\nobb.currency.price.historical(symbol='EURUSD', start_date='2023-01-01', end_date='2023-12-31', provider='fmp')\n# Get data with different granularity.\nobb.currency.price.historical(symbol='EURUSD', provider='polygon', interval=15m)\n```\n\n", metadata={'metadata': '{\'name\': \'obb_currency_price_historical\', \'description\': \'Currency Historical Price. Currency historical d